In [1]:
import torch
import numpy as np
import pandas as pd
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [2]:
# check if cuda is available
print(torch.cuda.is_available())
print(torch.version.cuda)

True
11.8


In [3]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [4]:
MODEL_NAME = "IlyaGusev/saiga_mistral_7b_lora"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Imagine you are a advertising copy writer and you are writing compelling copy for a new advertising campaign. You are writing personalized ad copy to a potential customer, but the copy should not be too creepy. The copy should be engaging and persuasive."

In [5]:
class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

In [6]:
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.bfloat16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

bin d:\Documents\GitHub\jarvis-ai-demo\.venv\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118_nocublaslt.dll


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

d:\Documents\GitHub\jarvis-ai-demo\.venv\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

d:\Documents\GitHub\jarvis-ai-demo\.venv\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



In [7]:
vehicle_list = ['Tesla Model S a dual motor all-wheel drive sedan with a 405-mile range, a top speed of 130 mph, and a 0-60 mph time of 3.1 seconds.',
                'Tesla Model S Plaid a tri motor all-wheel drive sedan with a 359-mile range, a top speed of 200 mph, and a 0-60 mph time of 1.99 seconds.',
                'Tesla Model 3 a real-wheel drive sedan with a 272-mile range, a top speed of 125 mph, and a 0-60 mph time of 5.8 seconds.',
                'Tesla Model 3 Long Range a dual motor all-wheel drive sedan with a 341-mile range, a top speed of 125 mph, and a 0-60 mph time of 4.2 seconds.',
                'Tesla Model X a dual motor all-wheel drive SUV with a 335-mile range, a top speed of 149 mph, and a 0-60 mph time of 3.8 seconds.',
                'Tesla Model X Plaid a tri motor all-wheel drive SUV with a 326-mile range, a top speed of 149 mph, and a 0-60 mph time of 2.5 seconds.',
                'Tesla Model Y a rear-wheel drive SUV with a 260-mile range, a top speed of 135 mph, and a 0-60 mph time of 6.6 seconds.',
                'Tesla Model Y Long Range a dual motor all-wheel drive SUV with a 330-mile range, a top speed of 135 mph, and a 0-60 mph time of 4.8 seconds.',
                'Tesla Model Y Performance a dual motor all-wheel drive SUV with a 303-mile range, a top speed of 155 mph, and a 0-60 mph time of 3.5 seconds.',
                'Tesla Cybertruck a dual motor all-wheel drive truck with a 300-mile range, a top speed of 130 mph, and a 0-60 mph time of 4.5 seconds.',
           ]

ads_location_list = ['WSJ.com',
                     'espn.com',
                     'washingtonpost.com',
                     'thecut.com',
                     'nypost.com',
                     'nytimes.com',
                     'cnn.com',
                     'huffpost.com',
                     'buzzfeednews.com',
                     'bloomberg.com',
                     ]

target_context_list = ['the person reading the ad is related to the person making the buying decision',
                       'the person reading the ad is making the buying decision']

ads_context_list = ['the ad is next to an article about finance',
                    'the ad is next to an article about sports',
                    'the ad is next to an article about politics',
                    'the ad is next to an article about fashion',
                    'the ad is next to an article about the weather']

target_profile_list = ['is a current Tesla owner', 
                       'has small children', 
                       'frequently visits restaurants',
                       'spends their weekends in national parks', 
                       'is an online shopper',
                       'is a business owner',
                       'is a college student',
                       'is retired',
                       'is a homeowner',
                       'lives in the country',
                       'lives in the suburbs',
                       'lives in the city',
                       'has a pet (dog)', 
                       'has a pet (cat)',
                       'have a gym membership',
                       'frequently travels', 
                       'has a large family', 
                       'is a grandparent', 
                       'has teenage children',
                       'has a garage', 
                       'likes to entertain friends and family',
                       'is a foodie',
                       'frequently visits concerts',
                       'frequently visits sporting events', 
                       'is geneneration x', 
                       'is a millennial', 
                       'is a baby boomer']

In [8]:
def get_ad(k=3):
    vehicle_to_advertise = np.random.choice(vehicle_list, size=1, replace=False).tolist()
    ad_location = np.random.choice(ads_location_list, size=1, replace=False).tolist()
    target_context = np.random.choice(target_context_list, size=1, replace=False).tolist()
    ad_context = np.random.choice(ads_context_list, size=1, replace=False).tolist()
    target_profile = np.random.choice(target_profile_list, size=min(len(target_profile_list), k), replace=False).tolist()
    target_profile = target_profile if np.random.random() > 0.2 else ['']
    target_context = target_context if np.random.random() > 0.2 else ['']

    text = f"""
    Imagine you are a advertising copy writer and you are writing compelling copy for a new advertising campaign. 
    You are writing personalized ad copy to a potential customer, but the copy should not be too creepy. 
    The copy should be engaging and persuasive.

    - Vehicle: {', '.join(vehicle_to_advertise)}
    - Ad appears on: {', '.join(ad_location)}
    - Ad context: {', '.join(ad_context)}
    - Potential customer context: {', '.join(target_context)}
    - Profile of the potential customer: {', '.join(target_profile)}

    Create a concise and compelling ad copy for a digital banner ad targeting the potential customer. 
    Focus on the potential customer's interests and preferences, taking into account his profile if it is specified.
    The consumer is aware of the vehicle and its features, so there is no need to describe the vehicle in detail.
    The ad should be concise and contain 1-2 sentences with no more than 150 characters.
    """
    short_text = f"""
    Create an attractive and concise ad focusing on the interest of the potential customer: {', '.join(target_profile)}

    Advertising information:
    {', '.join(vehicle_to_advertise)}
    {', '.join(ad_location)}
    {', '.join(ad_context)}
    {', '.join(target_context)}
    """
    return text, short_text

In [9]:
ads = get_ad()

In [10]:
print(ads[0])
print(ads[1])


    Imagine you are a advertising copy writer and you are writing compelling copy for a new advertising campaign. 
    You are writing personalized ad copy to a potential customer, but the copy should not be too creepy. 
    The copy should be engaging and persuasive.

    - Vehicle: Tesla Model Y Long Range a dual motor all-wheel drive SUV with a 330-mile range, a top speed of 135 mph, and a 0-60 mph time of 4.8 seconds.
    - Ad appears on: nytimes.com
    - Ad context: the ad is next to an article about politics
    - Potential customer context: the person reading the ad is making the buying decision
    - Profile of the potential customer: frequently visits restaurants, lives in the suburbs, is geneneration x

    Create a concise and compelling ad copy for a digital banner ad targeting the potential customer. 
    Focus on the potential customer's interests and preferences, taking into account his profile if it is specified.
    The consumer is aware of the vehicle and its featur

In [11]:
%%time
text_list = []
output_list = []
short_text_list = []
for _ in range(250):
    conversation = Conversation()
    text, short_text = get_ad()
    conversation.add_user_message(text)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print('---Prompt---')
    print(text)
    print('---Ad---')
    print(output)
    print()
    print("==============================")
    print()
    text_list.append(text)
    output_list.append(output)
    short_text_list.append(short_text)

d:\Documents\GitHub\jarvis-ai-demo\.venv\lib\site-packages\bitsandbytes\autograd\_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


---Prompt---

    Imagine you are a advertising copy writer and you are writing compelling copy for a new advertising campaign. 
    You are writing personalized ad copy to a potential customer, but the copy should not be too creepy. 
    The copy should be engaging and persuasive.

    - Vehicle: Tesla Model 3 a real-wheel drive sedan with a 272-mile range, a top speed of 125 mph, and a 0-60 mph time of 5.8 seconds.
    - Ad appears on: espn.com
    - Ad context: the ad is next to an article about politics
    - Potential customer context: the person reading the ad is related to the person making the buying decision
    - Profile of the potential customer: is geneneration x, is a baby boomer, is a current Tesla owner

    Create a concise and compelling ad copy for a digital banner ad targeting the potential customer. 
    Focus on the potential customer's interests and preferences, taking into account his profile if it is specified.
    The consumer is aware of the vehicle and its fe

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame({'prompt': text_list, 'ad': output_list, 'short_text': short_text})
df.to_csv('data.csv', index=False)
df

In [ ]:
# create json file for the data
import json

data = []
for i, row in df.iterrows():
    data.append({
        "prompt": row['prompt'],
        "ad": row['ad'],
        "short_text": row['short_text']
    })

In [ ]:
# save json to file and prettify it

with open('data.json', 'w') as f:
    json.dump(data, f, indent=4)
    